<a href="https://colab.research.google.com/github/Somixx21/Raport-10/blob/main/Raport_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
import csv
!wget https://github.com/Somixx21/Raport-10/blob/main/googleplaystore.csv
!wget https://dq-content.s3.amazonaws.com/350/AppleStore.csv
def load_csv(filename):
    with open(filename, encoding='utf-8') as f:
        reader = csv.reader(f)
        return list(reader)

google_data = load_csv('googleplaystore.csv')
apple_data = load_csv('AppleStore.csv')

--2025-05-23 12:27:24--  https://github.com/Somixx21/Raport-10/blob/main/googleplaystore.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘googleplaystore.csv.1’

googleplaystore.csv     [   <=>              ]   1.56M  3.20MB/s    in 0.5s    

2025-05-23 12:27:25 (3.20 MB/s) - ‘googleplaystore.csv.1’ saved [1637816]

--2025-05-23 12:27:25--  https://dq-content.s3.amazonaws.com/350/AppleStore.csv
Resolving dq-content.s3.amazonaws.com (dq-content.s3.amazonaws.com)... 52.217.233.121, 3.5.25.111, 16.182.109.169, ...
Connecting to dq-content.s3.amazonaws.com (dq-content.s3.amazonaws.com)|52.217.233.121|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 725761 (709K) [text/csv]
Saving to: ‘AppleStore.csv.1’

AppleStore.csv.1    100%[===================>] 708.75K  2.31MB/s    in 0.3s    

2025-05-23 12:27:26

In [ ]:
def only_good_length_rows(dataset, expected_length):
    return [row for row in dataset if len(row) == expected_length]

def remove_header(dataset):
    return dataset[1:]
def remove_duplicates(dataset, name_index):
    seen = {}
    for app in dataset:
        name = app[name_index]
        try:
            n_reviews = float(app[3])
        except:
            n_reviews = 0
        if name in seen:
            try:
                if n_reviews > float(seen[name][3]):
                    seen[name] = app
            except:
                pass
        else:
            seen[name] = app
    return list(seen.values())

def is_english(string):
    non_ascii = sum(1 for c in string if ord(c) > 127)
    return non_ascii <= 3

def filter_english_apps(dataset, name_index):
    return [app for app in dataset if is_english(app[name_index])]

def filter_free_apps(dataset, price_index):
    return [app for app in dataset if app[price_index] == '0.0' or app[price_index] == '0']

In [ ]:
def google_ready():
    step1 = only_good_length_rows(google_data, 13)
    step2 = remove_header(step1)
    step3 = remove_duplicates(step2, 0)
    step4 = filter_english_apps(step3, 0)
    step5 = filter_free_apps(step4, 7)
    return step5

In [ ]:
def apple_ready():
    step1 = only_good_length_rows(apple_data, 16)
    step2 = remove_header(step1)
    step3 = filter_english_apps(step2, 1)
    step4 = filter_free_apps(step3, 4)
    return step4

In [ ]:
google_clean = google_ready()
apple_clean = apple_ready()

print(f'Google Play Apps (clean): {len(google_clean)}')
print(f'Apple Store Apps (clean): {len(apple_clean)}')

# Przykładowe 2 aplikacje
print('\nGoogle Example:', google_clean[0])
print('\nApple Example:', apple_clean[0])

Google Play Apps (clean): 8864
Apple Store Apps (clean): 3222

Google Example: ['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']

Apple Example: ['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


In [ ]:
#KROK 12_2
def step12_2(dataset):
    from collections import defaultdict

    genre_totals = defaultdict(int)
    genre_counts = defaultdict(int)

    for app in dataset:
        genre = app[11]
        try:
            ratings = int(app[5])
            genre_totals[genre] += ratings
            genre_counts[genre] += 1
        except:
            continue

    genre_avgs = {}
    for genre in genre_totals:
        avg = genre_totals[genre] / genre_counts[genre]
        genre_avgs[genre] = int(round(avg, 0))

    return sorted(genre_avgs.items(), key=lambda x: x[1], reverse=True)

In [ ]:
def unich_step12_2():
    unich = '''
    Social Networking : 71548.34905660378
    Photo & Video : 28441.54375
    Games : 22788.6696905016
    Music : 57326.530303030304
    Reference : 74942.11111111111
    Health & Fitness : 23298.015384615384
    Weather : 52279.892857142855
    Utilities : 18684.456790123455
    Travel : 28243.8
    Shopping : 26919.690476190477
    News : 21248.023255813954
    Navigation : 86090.33333333333
    Lifestyle : 16485.764705882353
    Entertainment : 14029.830708661417
    Food & Drink : 33333.92307692308
    Sports : 23008.898550724636
    Book : 39758.5
    Finance : 31467.944444444445
    Education : 7003.983050847458
    Productivity : 21028.410714285714
    Business : 7491.117647058823
    Catalogs : 4004.0
    Medical : 612.0
    '''
    dct = {}
    for line in unich.strip().splitlines():
        parsed = line.split(':')
        genre = parsed[0].strip()
        average = float(parsed[1].strip())
        dct[genre] = int(round(average, 0))
    return sorted(dct.items(), key=lambda x: x[1], reverse=True)

In [ ]:
def verify_step12_2():
    ref = unich_step12_2()
    actual = step12_2(apple_clean)

    same = ref == actual
    print("Czy wyniki są zgodne:", same)

    if not same:
        print("\nRóżnice w kategoriach:")
        for (r_genre, r_val), (a_genre, a_val) in zip(ref, actual):
            print(f"{r_genre:<20} | expected: {r_val:<6} | got: {a_val:<6}")
    else:
        print("\nWyniki:")
        for genre, avg in actual:
            print(f"{genre:<20} : {avg}")

In [ ]:
verify_step12_2()

Czy wyniki są zgodne: True

Wyniki:
Navigation           : 86090
Reference            : 74942
Social Networking    : 71548
Music                : 57327
Weather              : 52280
Book                 : 39758
Food & Drink         : 33334
Finance              : 31468
Photo & Video        : 28442
Travel               : 28244
Shopping             : 26920
Health & Fitness     : 23298
Sports               : 23009
Games                : 22789
News                 : 21248
Productivity         : 21028
Utilities            : 18684
Lifestyle            : 16486
Entertainment        : 14030
Business             : 7491
Education            : 7004
Catalogs             : 4004
Medical              : 612


In [ ]:
#KROK 13.2
def step13_2(dataset):
    from collections import defaultdict

    category_totals = defaultdict(int)
    category_counts = defaultdict(int)

    for app in dataset:
        category = app[1]
        installs = app[5]
        try:
            installs_clean = installs.replace('+', '').replace(',', '')
            installs_int = int(installs_clean)
            category_totals[category] += installs_int
            category_counts[category] += 1
        except:
            continue

    category_avgs = {}
    for category in category_totals:
        avg = category_totals[category] / category_counts[category]
        category_avgs[category] = int(round(avg, 0))

    return sorted(category_avgs.items(), key=lambda x: x[1], reverse=True)

In [ ]:
def show_step13_2():
    results = step13_2(google_clean)
    print("Średnia liczba instalacji:\n")
    for category, avg in results:
        print(f"{category:<20} : {avg}")

In [ ]:
show_step13_2()

Średnia liczba instalacji:

COMMUNICATION        : 38456119
VIDEO_PLAYERS        : 24727872
SOCIAL               : 23253652
PHOTOGRAPHY          : 17840110
PRODUCTIVITY         : 16787331
GAME                 : 15588016
TRAVEL_AND_LOCAL     : 13984078
ENTERTAINMENT        : 11640706
TOOLS                : 10801391
NEWS_AND_MAGAZINES   : 9549178
BOOKS_AND_REFERENCE  : 8767812
SHOPPING             : 7036877
PERSONALIZATION      : 5201483
WEATHER              : 5074486
HEALTH_AND_FITNESS   : 4188822
MAPS_AND_NAVIGATION  : 4056942
FAMILY               : 3695642
SPORTS               : 3638640
ART_AND_DESIGN       : 1986335
FOOD_AND_DRINK       : 1924898
EDUCATION            : 1833495
BUSINESS             : 1712290
LIFESTYLE            : 1437816
FINANCE              : 1387692
HOUSE_AND_HOME       : 1331541
DATING               : 854029
COMICS               : 817657
AUTO_AND_VEHICLES    : 647318
LIBRARIES_AND_DEMO   : 638504
PARENTING            : 542604
BEAUTY               : 513152
EVENTS  